<h1 align="center">การใช้งาน Transaction Trino</h1>

<img src="https://miro.medium.com/v2/resize:fit:396/1*zeo2xk4tXiLbFRV8dbi3yg.png" width="200">
<h3 style="color: blue;" align="right">Author: Pasit Y.</h3>

---
> ### Function นี้รองรับเฉพาะ Hive >= 3 เท่านั้น
## Step Hive Enable ACID Transactions
#### Hive-site.xml Setting (Ambari or Cloudera)
```bash
Create Tables as ACID Insert Only = False (On Ambari)
hive.support.concurrency = true
hive.enforce.bucketing = true
hive.create.as.external.legacy = true
hive.exec.dynamic.partition.mode = nonstrict
hive.txn.manager = org.apache.hadoop.hive.ql.lockmgr.DbTxnManager
Trino Catalog > /home/trino/etc/catalog/hive.properties
Add Below Line `hive.storage-format=ORC`
```
---
## สร้าง Table ที่ Trino
---
```sh
CREATE TABLE hive.transaction.example (
    _c0 integer,
    vaccine_status varchar,
    outcome varchar,
    age_group varchar
)
WITH (
  format = 'ORC',
  transactional=true,
  partitioned_by = ARRAY['age_group'],
  bucketed_by = ARRAY['outcome'],
  bucket_count = 5
);
```
## ติดตั้ง Library
---
```py
pip install pandas
pip install trino
pip install trino[sqlalchemy]
```

In [1]:
import pandas as pd
from trino.dbapi import connect
import os
from sqlalchemy import create_engine
from sqlalchemy.schema import Table, MetaData
from sqlalchemy.sql.expression import select, text
import numpy as np

## Trino Connect with SQLAlchemy, Trino API

In [2]:
engine = create_engine('trino://hi@192.168.10.210:8090/iceberg/iceberg_test')
cur = engine.connect()

In [27]:
df = pd.read_csv("example.csv")
df = df.replace(' ', '_', regex=True)
df = df.rename(columns={'Unnamed: 0': 'ids'})

In [29]:
df.to_sql("nooknaja2", con=engine)

ProgrammingError: (trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=SYNTAX_ERROR, message="line 1:8: mismatched input 'INDEX'. Expecting: 'MATERIALIZED', 'OR', 'ROLE', 'SCHEMA', 'TABLE', 'VIEW'", query_id=20230326_105307_00151_68m82)
[SQL: CREATE INDEX ix_nooknaja2_index ON nooknaja2 (index)]
(Background on this error at: https://sqlalche.me/e/14/f405)

## ในกรณีต้องการสร้าง Auto Increment เพื่อ +1 ID ทุกครั้งในการ Loop Insert

In [ ]:
max_id = pd.read_sql("SELECT MAX(idx) as max_id FROM hive.transaction.example", con=engine)

id_max = 0

if max_id['max_id'][0] is None:
    id_max = 0
else:
    id_max = max_id['max_id'][0]

## Loop inserting row-by-row Using Transaction ACID

In [ ]:
df2 = df[20:40]
i = id_max
for row in df2.itertuples():
    i = i + 1
    sql = f"""
            INSERT INTO
                hive.transaction.example
            VALUES (
            	{row.ids},'{row.vaccine_status}','{row.outcome}',{i},'{row.age_group}'
            )
    """
    cur.execute(sql)
    print('Insert Rows :[', row.age_group,']', 'IDx:[', i,']')

## Select Data

In [ ]:
cur.execute("SELECT * FROM transaction.example").fetchall()

## Update Data

In [ ]:
cur.execute("UPDATE transaction.example SET vaccine_status = 'Test Values' WHERE _c0 = 3").fetchone()

## Delete Data

In [ ]:
cur.execute("DELETE FROM transaction.example WHERE _c0 = 1").fetchone()